# Investigating performance and scaling of CFDPINN and OpenFOAM

OpenFOAM is a highly optimized computational fluid dynamics software package.

In this notebook we compare and contrast OpenFOAM scalability against CFDPINN.

We begin by examining the simulation time using OpenFOAM for three examples of the lid-driven cavity case. These three cases use differing mesh sizes which in turn impacts the time to solution. The number of timesteps is held constant, as is the initial conditions.

- 20x20
- 100x100
- 200x200

We can then take the trained CFDPINN and perform inference for each of the three grid sizes.

- Load trained PINN model
- Setup features based on various geometries
- Perform inference with timing
- Compare accuracy

In [1]:
from cfdpinn.args import parse_args
import sys

sys.argv = [
    "program-name",
    "--case-type",
    "cavity",
    "--case-dir",
    "/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1_100x100",
    "--start-time",
    "0",
    "--end-time",
    "5",
    "--dt",
    "0.001",
    "--numx",
    "100",
    "--numy",
    "100",
    "--viscosity",
    "0.01",
    "--initial_u",
    "1",
    "--scaler-path",
    "/Users/harry/pinns/models/tmp-scaler.pkl",
    "--model-path",
    "/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt",
    "--epochs",
    "1",
    "--training-animation",
    "--prediction-animation",
    "--residual-animation",
    "--num-frames",
    "5",
    "--inference-timing",
    "--output-pred-data",
    "--output-train-data",
    "--output-data-path",
    "/Users/harry/pinns/data/tmp-outputs"
]

args_100x100 = parse_args()
print(args_100x100)


#################
##  CFD PINN   ##
#################

Namespace(case_type='cavity', case_dir='/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1_100x100', start_time=0.0, end_time=5.0, dt=0.001, numx=100, numy=100, viscosity=0.01, initial_u_lid=1.0, test_size=0.7, num_frames=5, training_animation=True, prediction_animation=True, residual_animation=True, static_plots=True, output_pred_data=True, output_train_data=True, output_data_path='/Users/harry/pinns/data/tmp-outputs', pinn_output_path='/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt', tensorboard=False, epochs=1, learning_rate=0.001, inference_timing=True, verbose=False)


In [2]:
from cfdpinn.args import parse_args
import sys

sys.argv = [
    "program-name",
    "--case-type",
    "cavity",
    "--case-dir",
    "/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1",
    "--start-time",
    "0",
    "--end-time",
    "5",
    "--dt",
    "0.001",
    "--numx",
    "20",
    "--numy",
    "20",
    "--viscosity",
    "0.01",
    "--initial_u",
    "1",
    "--scaler-path",
    "/Users/harry/pinns/models/tmp-scaler.pkl",
    "--model-path",
    "/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt",
    "--epochs",
    "1",
    "--training-animation",
    "--prediction-animation",
    "--residual-animation",
    "--num-frames",
    "5",
    "--inference-timing",
    "--output-pred-data",
    "--output-train-data",
    "--output-data-path",
    "/Users/harry/pinns/data/tmp-outputs"
]

args_20x20 = parse_args()
print(args_20x20)


#################
##  CFD PINN   ##
#################

Namespace(case_type='cavity', case_dir='/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1', start_time=0.0, end_time=5.0, dt=0.001, numx=20, numy=20, viscosity=0.01, initial_u_lid=1.0, test_size=0.7, num_frames=5, training_animation=True, prediction_animation=True, residual_animation=True, static_plots=True, output_pred_data=True, output_train_data=True, output_data_path='/Users/harry/pinns/data/tmp-outputs', pinn_output_path='/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt', tensorboard=False, epochs=1, learning_rate=0.001, inference_timing=True, verbose=False)


In [4]:
#Load trained PINN (on 20x20)
import torch
pinn_20x20 = torch.load("/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt")
pinn_20x20

CfdPinn(
  (linear_stack): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=3, bias=True)
  )
  (criterion): MSELoss()
)

In [7]:
#Setup geom for 20x20 case
from cfdpinn.geometry import setup_geom

geom_20x20 = setup_geom(args_20x20)
geom_20x20

Setting up geometry...
	Geometry setup completed



{'x_start': 0,
 'x_end': 1,
 'numx': 20,
 'y_start': 0,
 'y_end': 1,
 'numy': 20,
 't_start': 0.0,
 't_end': 5.0,
 't_dt': 0.001,
 'numt': 5001,
 'grid2d_x': array([[0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
         0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
         0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
         0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ],
        [0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
         0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
         0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
         0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ],
        [0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
         0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
         0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
         0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ],
        [0. 

In [12]:
from cfdpinn.pinns import predict_fluid
from cfdpinn.preprocess import merge_features
from pickle import load

data = dict()

data = merge_features(data,geom_20x20)

data["scaler"] = load(open("/Users/harry/pinns/models/tmp-scaler.pkl","rb"))

predict_fluid(data,pinn_20x20,geom_20x20,args_20x20)

Prediction of fluid properties...

** Inference Timings **

DEVICE: cpu
	Median: 0.644676923751831
	Min: 0.586076021194458
	Max: 0.6873290538787842
	Std.dev: 0.029851868481131763



KeyError: 'u'